### Notebook to make Fast Point-Cloud Diffusion distributions Discrete, like G4

In [1]:
import h5py
import numpy as np
from tqdm import tqdm

In [2]:
#Must be a naturally discrete dataset, from G4!!!

#geant4_name = "improved_200cells_FPCD.hdf5"
geant4_name = "improvedMIP_200cells_FPCD.hdf5"
g4 = h5py.File(geant4_name, 'r')
print(g4.keys())
chunk_size=2000
nevts=100_000

<KeysViewHDF5 ['cluster', 'hcal_cells']>


In [3]:
def get_bin_edges(g4_cell_data):
    centers = np.unique(g4_cell_data)
    if (centers[0] == 0):
        centers = centers[1:]
    width = np.round(centers[1] - centers[0],2)

    edges = centers - width/2
    max_edge = centers[-1] + width/2
    edges = (np.append(edges,max_edge))
    
    return centers, edges

In [4]:
bin_dict = {}
var_str = ["E","X","Y","Z"]

for var in range(1,4):
    g4_data = g4['hcal_cells'][:nevts,:,var]
    centers, edges = get_bin_edges(g4_data)
    bin_dict[f"centers{var_str[var]}"],bin_dict[f"edges{var_str[var]}"] = centers, edges 

print(bin_dict.keys())

dict_keys(['centersX', 'edgesX', 'centersY', 'edgesY', 'centersZ', 'edgesZ'])


In [5]:
#Dataset of Continuous distributions, to be converted
g4_disc = False

dset_name = 'cell_features'
cluster_name = 'cluster_features'
discrete_name = "GSGM_Discrete.h5"
continuous_name = "GSGM.h5"

if (g4_disc):
    dset_name = 'hcal_cells' #for smeared G4 data, not for GSGM
    cluster_name = "cluster"
    discrete_name = "G4_Discrete.h5"
    continuous_name = "newMIP_smeared_20keV_200cells_FPCD.hdf5"


In [6]:
dfsn = h5py.File(continuous_name,'r')
print(dfsn.keys())

digit_dict = {}
var_str = ["E","X","Y","Z"]
for var in range(1,4):
    diffusion_data = dfsn[dset_name][:,:,var]
    digits = np.digitize(diffusion_data,bin_dict[f"edges{var_str[var]}"])
    print(var_str[var],": ",digits[100,:10])
    digit_dict[f"digits{var_str[var]}"] = digits - 1  # -1 for indices

<KeysViewHDF5 ['cell_features', 'cluster_features']>
X :  [19 18 19 19 18 19 19 19 19 20]
Y :  [18 19 19 19 19 20 20 19 20 20]
Z :  [13  6 16  4 14 21  1  5 10  4]


In [7]:
np.shape(dfsn[cluster_name][1])

(2,)

In [8]:
test = 0
nevents = np.shape(dfsn[dset_name])[0]
nevents = 10_000
ncells = np.shape(dfsn[dset_name])[1]
nvar = np.shape(dfsn[dset_name])[2]
ncluster_var = np.shape(dfsn[cluster_name])[1]
chunk_size = 100
with h5py.File(discrete_name, 'w') as newfile:
    # create empty data set
    dset = newfile.create_dataset(dset_name, 
                                shape=(np.shape(dfsn[dset_name])),
                                maxshape=(np.shape(dfsn[dset_name])), 
                                chunks=(chunk_size, ncells, nvar),
                                dtype=np.float32)

    cluster_dset = newfile.create_dataset(cluster_name, data=dfsn[cluster_name])
    
    
    dset[:,:,0] = dfsn[dset_name][:,:,0]
    dset[:,:,-1] = dfsn[dset_name][:,:,-1]
    for var in range(1,4):
        
        g4_centers = bin_dict[f"centers{var_str[var]}"]  # what the data will be set to
        n_bins = len(bin_dict[f"centers{var_str[var]}"]) 
        var_mask =  digit_dict[f"digits{var_str[var]}"]  #which data will be edited
        #print("var mask shape = ",np.shape(var_mask))
        diffusion_data = dfsn[dset_name][:,:,var]
        #print("var_mask = ",var_mask[10])
        for evt in tqdm(range(nevents)):
            for ibin in range(n_bins):
                bin_mask = var_mask[evt]==ibin
                #print("BIN NUMBER",ibin)
                #print("Var Mask sample",var_mask[evt,25:35])
                #print("Data = ",diffusion_data[evt,25:35][var_mask[evt,25:35]==ibin])
                #print("BIN CENTER = ",g4_centers[ibin])
                diffusion_data[evt][bin_mask] = g4_centers[ibin]
                #print("Data Discr= ",diffusion_data[evt,25:35][var_mask[evt,25:35]==ibin])
            
                
        dset[:,:,var] = np.round(diffusion_data,2)
        print(np.round(diffusion_data[100,25:35],2))
        print(f"Done with {var_str[var]}")

100%|██████████| 10000/10000 [00:00<00:00, 10482.68it/s]


[ -900.  -900. -1000.  -900.  -900.  -900. -1000.  -900. -1100.  -800.]
Done with X


100%|██████████| 10000/10000 [00:00<00:00, 10126.98it/s]


[ -900.  -900.  -900.  -900.  -900.  -800.  -800.  -800.  -900. -1000.]
Done with Y


100%|██████████| 10000/10000 [00:01<00:00, 9928.64it/s]

[4102.3 3961.9 4219.3 4289.5 3844.9 4055.5 4429.9 3938.5 4125.7 4219.3]
Done with Z


In [9]:
with h5py.File(f'G4_Discrete.h5', 'r') as disc:
    mask = disc["hcal_cells"][10000,:,-1] != 0
    cellX = disc["hcal_cells"][10000,:,1][mask]
    print(np.unique(cellX))

[-1652.46 -1449.17 -1367.29 -1248.4  -1243.31 -1234.24 -1198.72 -1177.31]


In [10]:
with h5py.File(f'GSGM_Discrete.h5', 'r') as disc:
    mask = disc["cell_features"][1000,:,-1] != 0
    cellX = disc["cell_features"][1000,:,1][mask]
    print(np.unique(cellX))
    

[ 800.  900. 1000. 1100. 1200. 1300.]


In [11]:
print(g4["hcal_cells"][100,:10,3])

[3821.5 3844.9 3868.3 3915.1 3938.5 3938.5 3891.7 3844.9 3868.3 3915.1]
